In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch


model_name = "aubmindlab/aragpt2-base"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


In [3]:
tokenizer.pad_token = tokenizer.eos_token

In [2]:
from datasets import load_dataset

dataset = load_dataset("oscar", "unshuffled_deduplicated_arz" ,cache_dir="./dataset")

In [ ]:
def tokenize_function(example):
    encodings = tokenizer(example['text'], truncation=True, padding="max_length", max_length=128)

    encodings['labels'] = encodings['input_ids'].copy()

    return encodings

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [10]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [11]:
train_size = int(0.8 * len(tokenized_datasets))  # 80% for training
train_dataset = tokenized_datasets.select(range(train_size))  # First 80% as training
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))  # Remaining 20% for validation

In [12]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./aragpt2_finetuned",  # Directory to save the model
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=4,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # Weight decay to prevent overfitting
    logging_dir="./logs",  # Directory for storing logs
    logging_steps=100,  # Log every 100 steps
    eval_strategy="epoch",  # Evaluate after each epoch
    save_steps=500,  # Save the model every 500 steps
    save_total_limit=2,  # Save only the last 2 models to save disk space
    fp16=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-12-2848cc39e083>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.112800,2.127221
2,1.965400,2.078082
3,1.775600,2.060662


TrainOutput(global_step=14388, training_loss=2.0420265208359125, metrics={'train_runtime': 2116.9679, 'train_samples_per_second': 27.183, 'train_steps_per_second': 6.797, 'total_flos': 3759077818368000.0, 'train_loss': 2.0420265208359125, 'epoch': 3.0})